#### Import needed packages

In [ ]:
import numpy as np # mathematical package
import matplotlib.pyplot as plt # plotting package
import cartopy.crs as ccrs # package for Map-plots
from netCDF4 import Dataset # package for reading in nc-files

#### Variables

In [ ]:
#Dictionary: {How we will call it : how it is called in the nc-file}
vars_SH={
    'time':'time', # Time [hours since 2015-12-3 00:00:00]
    'lon':'lon',   # Longitude [degrees_east]
    'lat':'lat',   # Latitude [degrees_north]
    'p':'plev',    # pressure [Pa]
    'u':'U',       # X-component of horizontal velocity [m s-1]
    'v':'V',       # Y-component of horizontal velocity [m s-1]
    'w':'W',       # Vertical velocity [Pa s-1]
    't':'T',       # Temperature [K]
    'rh':'R',      # Relative humidity [%]
    'gp':'Z',      # Geopotential height [m2 s-2]
    'vort':'VO',   # Relative Vorticity [s-1]  
    'div':'D',     # Divergence [s-1]
    'ps':'SP'      # Surface pressure [Pa]
}
vars_GG={
    'time':'time',  # Time [day as %Y%m%d.%f]
    'lon':'lon',   # Longitude [degrees_east]
    'lat':'lat',   # Latitude [degrees_north]
    'p':'plev',    # pressure [Pa]
    'shf':'SSHF',    # Surface sensible heat flux [Wm-2s]
    'lhf':'SLHF',    # Surface latent heat flux [Wm-2s]
    'pr':'TP',      # Total precipitation [m]
    'q':'Q',       # Specific humidity [kg kg-1]
    't2m':'T2M',    # 2 metre temperatur [K]
    'lc':'LCC',    # Low cloud cover [-]
    'mc':'MCC',  # Medium cloud cover [-]  
    'tc':'TCC',   # Total cloud cover [-]
    'tlw':'var78', # Total column liquid water [kg m-2] 
    'blh':'BLH'     # Boundary layer height [m]
}

#### Initialise data variables (Dictionaries)

In [ ]:
dat_c={} # control data
dat_ne={} # no evaporation on precipitation
dat_nr={} # no runoff
dat_ns={} # no interactive surface scheme

# Read in data

In [ ]:
ipath='/Data/gfi/work/ldi022/GEOF321/Desmond_T255L91/output/' # TODO: Insert the path of your model's output (ncfiles)

#### Read in data from **control run**

In [ ]:
filename='Desmond2013GG_control.nc' # TODO: Insert the file name of your model's control run output file (gaussian)
f=Dataset(ipath+filename) # Open ncfile
for var in vars_GG: # For all variables
    dat_c[var]=np.squeeze(f.variables[vars_GG[var]][:])
f.close()

filename='Desmond2013SH_control.nc' # TODO: Insert the file name of your model's control run output file (spherical harmonics)
f=Dataset(ipath+filename) # Open ncfile
for var in vars_SH: # For all variables
    dat_c[var]=np.squeeze(f.variables[vars_SH[var]][:])
f.close()

#### Read in data from **no evaporation on precipitation run**

In [ ]:
filename='Desmond2013GG_no_evap_on_prec.nc' # TODO: Insert the file name of your model's run without "evaporation on precipitation" output file (gaussian)
f=Dataset(ipath+filename) # Open ncfile
for var in vars_GG: # For all variables
    dat_ne[var]=np.squeeze(f.variables[vars_GG[var]][:])
f.close()

filename='Desmond2013SH_no_evap_on_prec.nc' # TODO: Insert the file name of your model's run without "evaporation on precipitation" output file (spherical harmonics)
f=Dataset(ipath+filename) # Open ncfile
for var in vars_SH: # For all variables
    dat_ne[var]=np.squeeze(f.variables[vars_SH[var]][:])
f.close()

#### Read in data from **no orographical runoff run**

In [ ]:
filename='Desmond2013GG_no_runoff.nc' # TODO: Insert the file name of your model's run without "evaporation on precipitation" output file (gaussian)
f=Dataset(ipath+filename) # Open ncfile
for var in vars_GG: # For all variables
    dat_nr[var]=np.squeeze(f.variables[vars_GG[var]][:])
f.close()

filename='Desmond2013SH_no_runoff.nc' # TODO: Insert the file name of your model's run without "evaporation on precipitation" output file (spherical harmonics)
f=Dataset(ipath+filename) # Open ncfile
for var in vars_SH: # For all variables
    dat_nr[var]=np.squeeze(f.variables[vars_SH[var]][:])
f.close()

#### Read in data from **no interactive surface scheme run** (Model crashed for this experiment)

In [ ]:
# filename='Desmond2013GG_no_surface_scheme.nc' # TODO: Insert the file name of your model's run without "evaporation on precipitation" output file (gaussian)
# f=Dataset(ipath+filename) # Open ncfile
# for var in vars_GG: # For all variables
#     dat_nr[var]=f.variables[vars_GG[var]][:]
# f.close()

# filename='Desmond2013SH_no_surface_scheme.nc' # TODO: Insert the file name of your model's run without "evaporation on precipitation" output file (spherical harmonics)
# f=Dataset(ipath+filename) # Open ncfile
# for var in vars_SH: # For all variables
#     dat_nr[var]=f.variables[vars_SH[var]][:]
# f.close()

# Plot results

In [ ]:
labels={'u':'U Wind [ms$^{-1}$]','v':'V Wind [m s$^{-1}$]','w':'Vertical Velocity [Pa s$^{-1}$]','t':'Temperature [K]',
        'rh':'Relative humidity [%]','gp':'Geopotential height [m2s$^{-2}$]','vort':'Relative Vorticity [s$^{-1}$]',
        'div':'Divergence [s$^{-1}$]','ps':'Surface Pressure [Pa]','shf':'Surface sensible heat flux [Wm$^{-2}$s]',
        'lhf':'Surface latent heat flux [Wm$^{-2}$s]','pr':'Total precipitation [m]','q':'Specific humidity [kg kg$^{-1}$]',
        't2m':'2m Temperature [K]','lc':'Low cloud cover','mc':'Medium cloud cover','tc':'Total cloud cover',
        'tlw':'Total column liquid water [kg m$^{-2}$]','blh':'Boundary layer height [m]'}

cols={'c':[0,0.5,0.7], 'ne':[], 'nr':[], 'ns':[]} #Colors for control (c), no evap. on prec. (ne), no orogr. runoff (nr), no interac. surface (ns)
fs=14 # Fontsize

## Plot functions

#### Plot function for Profiles

In [ ]:
def plot_profiles(data, p, lat, lon, lats, lons, title, fs=14,top=-6,legend=False):
    """ This function plots profiles averaged over a given lat-lon box.
        data: needs to have the dimensions (time, plev, lat, lon)
        p: vector of pressure
        lat,lon: Vector of Latitudes and Longitudes
        lats, lons: arrays [lower_boundary_degree, upper_boundary_degree] that define the boundaries of the lat-lon-box 
        title: needs to be a string
        fs: fontsize
        top: Highest vertical level of interest (-1 for all levels)
        legend: True if a legend should be printed, False to avoid a legend
    """
    lat_min=np.argmin(np.abs(lat-lats[1]))
    lat_max=np.argmin(np.abs(lat-lats[0]))
    lon_min=np.argmin(np.abs(lon-lons[0]))
    lon_max=np.argmin(np.abs(lon-lons[1]))
    plt.plot(np.mean(data[0,:top,lat_min:lat_max,lon_min:lon_max],axis=(1,2)),p[:top]/100,label='Initial profile',lw=2) # Plot initial profile
    plt.plot(np.mean(data[24,:top,lat_min:lat_max,lon_min:lon_max],axis=(1,2)),p[:top]/100,label='Profile after 3 days',lw=2) # Plot profile after 24 output timesteps (3 days)
    plt.gca().invert_yaxis() # inverts yaxis
    if legend:
        plt.legend(fontsize=fs,handlelength=1,bbox_to_anchor=(1.05,1))
    plt.xlabel(labels[var],fontsize=fs)
    plt.ylabel('Pressure [hPa]',fontsize=fs)
    plt.title(title,fontsize=fs)

#### Plot function for Maps

In [ ]:
def plot_maps(data, lat, lon, lats, lons, title, vmin=None, vmax=None, cmap='seismic', luts=11,fs=14):
    """ This function plots maps of any 2 dimensional variable.
        data: needs to have the dimensions (lat, lon)
        lat,lon: Vector of Latitudes and Longitudes
        lats,lons: Vector [lower_boundary_degree, upper_boundary_degree] that define the boundaries of the lat-lon-box 
        title: Needs to be a string
        vmin, vmax: Minimum and Maximum of the colorscale
        cmap: Colormap, e.g. 'OrRd','seismic','viridis','terrain', look up in the internet
        luts: Number of discrete sperations in coloscale, "None" for continuous scale
        fs: fontsize
    """
    if vmin==None: #If no max and min values are given, set to 5th and 95th percentile
        vmin=np.percentile(data,5)
    if vmax==None:
        vmax=np.percentile(data,95)
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=7))
    ax.coastlines()
    map1 = ax.pcolormesh(lon, lat, data, transform=ccrs.PlateCarree(),cmap=plt.get_cmap(cmap,lut=luts),vmin=vmin,vmax=vmax)
    cbar = plt.colorbar(map1,orientation='vertical',aspect=20,shrink=0.70,pad=0.03)#,ax=ax)
    cbar.ax.tick_params(labelsize=fs-2)
    ax.set_extent([lons[0],lons[1],lats[0],lats[1]], crs=ccrs.PlateCarree())
    plt.title(title,fontsize=fs)

## Create Plots

#### Plot profiles (averaged over the lat-lon-box of interest)

In [ ]:
lats=[50,62] # TODO: Cut off the lat-lon box of interest in degree
lons=[0,15]  # TODO: Cut off the lat-lon box of interest in degree # Minimum is unfortunately 0
opath='/Data/gfi/work/ldi022/GEOF321/Desmond_T255L91/plots/'

# Possible Variables for Profile-Plot: q, u, v, w, t, rh, gp, vort, div
for var in ['t','rh','w','vort','div']: # Loop over all mentioned variables #TODO: For which variables do you need profiles?
    plt.figure(figsize=(15,3))
    plt.subplot(141)
    plot_profiles(dat_c[var],dat_c['p'],dat_c['lat'],dat_c['lon'],lats,lons,'Control')
    plt.subplot(142)
    plot_profiles(dat_ne[var],dat_c['p'],dat_c['lat'],dat_c['lon'],lats,lons,'No evaporation')
    plt.subplot(143)
    plot_profiles(dat_ne[var]-dat_c[var][:25],dat_c['p'],dat_c['lat'],dat_c['lon'],lats,lons,'Difference')
    plt.subplot(144)
    plot_profiles(dat_ne[var]-dat_c[var][:25],dat_c['p'],dat_c['lat'],dat_c['lon'],[-90,90],[0,360],'Difference, globally',legend=True)
    plt.subplots_adjust(wspace=0.35)
    #plt.savefig(opath+var+'_profile_comparison_no_evaporation.png',dpi=120,bbox_inches='tight')

#### Plot Maps

#### Plot 3D Atmospheric Data

In [ ]:
lats=[50,80] # TODO: Cut off the lat-lon box of interest in degree
lons=[-30,30]  # TODO: Cut off the lat-lon box of interest in degree

##### Atmospheric Data
# Possible Variables for Map-Plot in any height: q, u, v, w, t, rh, gp, vort, div
var='vort' # Variable in strings
ts=24 # timestep
hl=4 # height level (0: 1000hPa, 2: 850hPa, 4: 500hPa, 8: 200hPa, 21 (top): 1hPa)

plot_maps(dat_ne[var][ts,hl]-dat_c[var][ts,hl],dat_c['lat'],dat_c['lon'],lats,lons,labels[var],vmin=-3e-4,vmax=3e-4,luts=13)
opath='/Data/gfi/work/ldi022/GEOF321/Desmond_T255L91/plots/'
plt.savefig(opath+var+'_map_comparison_no_evaporation.png',dpi=120,bbox_inches='tight')

#### Plot 2D Surface Data

In [ ]:
lats=[50,80] # TODO: Cut off the lat-lon box of interest in degree
lons=[-30,30]  # TODO: Cut off the lat-lon box of interest in degree

##### Surface Data
# Possible Variables for Map-Plot at the surface: shf, lhf, pr, t2m, lc, mc, tc, tlw, blh, ps
var='blh' # Variable in strings
ts=24 # timestep
hl=4 # height level (0: 1000hPa, 2: 850hPa, 4: 500hPa, 8: 200hPa, 21 (top): 1hPa)
plt.figure()
plot_maps(dat_ne[var][ts]-dat_c[var][ts],dat_c['lat'],dat_c['lon'],lats,lons,labels[var],luts=13)
opath='/Data/gfi/work/ldi022/GEOF321/Desmond_T255L91/plots/'
plt.savefig(opath+var+'_map_comparison_no_evaporation.png',dpi=120,bbox_inches='tight')